In [ ]:
import os
import pandas as pd

# Specify the folder path containing the CSV files
folder_path = '/content/drive/MyDrive/validation_clean'

# Initialize an empty list to store DataFrames
dfs = []

# Loop through each file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(folder_path, filename)

        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path)
        for feature in ['access_success_rate', 'data_rate', 'resource_utilition_rate', 'TA', 'mcs', 'bler', 'cqi']:
            for i in range(1, 2):
                df[feature] = df[feature].shift(i)

        # Add a new column with the filename as a feature
        df['file_name'] = filename

        # Append the modified DataFrame to the list
        dfs.append(df)

# Concatenate all DataFrames into a single DataFrame
result_df999 = pd.concat(dfs, ignore_index=True)

In [ ]:
import os
import pandas as pd

# Specify the folder path containing the CSV files
folder_path = '/content/drive/MyDrive/train_test_dataset_Fault Impact Analysis'

# Initialize an empty list to store DataFrames
dfs = []

# Loop through each file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(folder_path, filename)

        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path)

        # Perform operations on the DataFrame
        df['data_rate_change'] = df['data_rate'] - df['data_rate'].shift(1)
        df['data_rate_change'] = df['data_rate_change'].apply(lambda x: 0 if x > 0 else 1)
        for feature in ['access_success_rate', 'data_rate', 'resource_utilition_rate', 'TA', 'mcs', 'bler', 'cqi']:
            for i in range(1, 2):
                df[feature] = df[feature].shift(i)
        index = df[df['fault_duration'] > 0].index[0]
        rows_before = df.iloc[:index]
        target_row = df.iloc[index]
        result = pd.concat([rows_before, target_row], ignore_index=True)
        dfs.append(df)

# Concatenate all DataFrames into a single DataFrame
result_df = pd.concat(dfs, ignore_index=True)

In [ ]:
result_df9=result_df999[result_df999['fault_duration']>0]
#result_df99=result_df[result_df['fault_duration']>0]
#result_df9.head()

In [ ]:
import numpy as np
import pandas as pd
#from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import KFold,StratifiedKFold ,GroupKFold
from sklearn.metrics import mean_absolute_error,f1_score,mean_squared_error,roc_auc_score
import lightgbm as lgb
from sklearn.preprocessing import PolynomialFeatures
import warnings
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans
from sklearn import preprocessing
warnings.filterwarnings('ignore')
from xgboost import XGBClassifier
import tqdm
from tqdm import tqdm_notebook
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
train=result_df#pd.read_csv('tttt.csv')
train['data_rate_change'].fillna(0,inplace=True)
test=result_df9
train['win']=1
test['win']=0
data=pd.concat([train,test]).reset_index(drop=True)
data.fillna(df.groupby(['NE ID','endTime'], as_index=False).mean(), inplace=True)
data.fillna(df.mean(), inplace=True)
data["time"] = pd.to_datetime(data["endTime"])
data["month"] = data.time.dt.month
data["year"] =  data.time.dt.year
data["day"] = data.time.dt.day
data["hour"] = data.time.dt.hour
data['week']=data.time.dt.weekofyear
data['dayofyear']=data.time.dt.dayofyear
data['dayofweek']=data.time.dt.dayofweek
data["is_month_end"] = data.time.dt.is_month_end
data["is_quarter_end"] = data.time.dt.is_quarter_end
data["is_year_end"] = data.time.dt.is_year_end
data['hour_sin'] = np.sin(2 * np.pi * data['hour'] / 24)
data['hour_cos'] = np.cos(2 * np.pi * data['hour'] / 24)
data['day_sin'] = np.sin(2 * np.pi * data['dayofweek'] / 7)
data['day_cos'] = np.cos(2 * np.pi * data['dayofweek'] / 7)
data['mont_sin'] = np.sin(2 * np.pi * data['month'] / 12)
data['mont_cos'] = np.cos(2 * np.pi * data['month'] / 12)
###
engFeature=[ 'fault_duration', 'relation']



train=data[data['win']==1]
test=data[data['win']==0]
feature=[c for c in train.columns if c not in ['kfold','time','0','ID','endTime','NE ID','data_rate_t+1_trend','data_rate_change','file_name','Unnamed: 0']]

In [ ]:
df=train
df['kfold']=-1
df=df.sample(frac=1,random_state=1998).reset_index(drop=True)#6,7,11
kf=KFold(n_splits=5)
for fold,(trn_,val_) in enumerate(kf.split(X=df)):
    df.loc[val_,'kfold']=fold
df=df
from sklearn.linear_model import LinearRegression
final_predictions=[]
scores=[]#1
for fold in range(5):
    xtrain=df[df.kfold != fold].reset_index(drop=True)
    xvalid=df[df.kfold == fold].reset_index(drop=True)
    xtest=test[feature]
    ytrain=xtrain.data_rate_change
    yvalid=xvalid.data_rate_change
    xtrain=xtrain[feature]
    xvalid=xvalid[feature]
    model=LGBMClassifier(scale_pos_weight=1.5,n_estimators=500,learning_rate=0.05)#n_estimators=4000,learning_rate=0.01)
    model.fit(xtrain,ytrain)#early_stopping_rounds=150,eval_set=[(xvalid,yvalid)],verbose=500)
    preds_valid=model.predict(xvalid)
    predictions=model.predict(xtest)
    final_predictions.append(predictions)
    rmse=f1_score(yvalid,preds_valid)
    print((f'fold {fold} result =='),rmse)
    scores.append(rmse)
print(np.mean(scores),np.std(scores))

[LightGBM] [Info] Number of positive: 379543, number of negative: 347594
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.180049 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2346
[LightGBM] [Info] Number of data points in the train set: 727137, number of used features: 22
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.521969 -> initscore=0.087933
[LightGBM] [Info] Start training from score 0.087933
fold 0 result == 0.7233184211220194
[LightGBM] [Info] Number of positive: 379181, number of negative: 347956
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.125253 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2347
[LightGBM] [Info] Number of data points in the train set: 727137, number of used features: 22
[LightGBM] [Info] [binary:BoostFromS

In [ ]:
test['data_rate_change']=np.mean(np.column_stack(final_predictions),axis=1)#model.predict(test[feature])#
test['ID']=test['file_name'].apply(lambda x:x.split('.')[0])
test['data_rate_change'] = [1 if x >= 0.61 else 0 for x in test['data_rate_change']]
test[['ID','data_rate_change']].to_csv('fault_winn1.csv',index=False)